In [1]:
!pip install openai

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 227.4/227.4 kB 4.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 75.6/75.6 kB 10.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 77.8/77.8 kB 7.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 58.3/58.3 kB 7.4 MB/s eta 0:00:00


In [4]:
import openai
import getpass
# Prompt the user for the API key
api_key = getpass.getpass("Enter your OpenAI API key: ")

# Assign the API key to the OpenAI client
openai.api_key = api_key

Enter your OpenAI API key: ··········


In [5]:
from openai import OpenAI

In [6]:
client = OpenAI(
    # defaults to os.environ.get("OPENAI_API_KEY")
    api_key=api_key,
)

In [11]:
def generate(prompt):
    """
    Initializes a chat with GPT-4 and sends a prompt to get a response.

    Args:
    prompt (str): The prompt or question to send to GPT-4.

    Returns:
    str: The content of GPT-4's response.
    """
    try:
        openai.api_key = api_key
        completion = client.chat.completions.create(
            model="gpt-4",  # gpt-4
            messages=[
                {
                    "role": "user",
                    "content": prompt,
                },
            ],
        )
        return completion.choices[0].message.content
    except Exception as e:
        return str(e)

In [23]:
def validate_weight(weight):
    try:
        weight = float(weight)
        if weight <= 0:
            raise ValueError
    except ValueError:
        return False
    return True

def get_gender_choice():
    return {1: 'Male', 2: 'Female'}

def get_dietary_restrictions():
    return {
        1: 'None',
        2: 'Keto',
        3: 'Vegan',
        4: 'Vegetarian',
        5: 'Gluten-Free',
        6: 'Paleo',
        7: 'Halal',
        8: 'Other'
    }

def get_goal():
    return {
        1: 'Gain Weight',
        2: 'Lose Weight',
        3: 'Maintain Weight'
    }

def input_user_data():
    weight = input("Enter your weight in lbs: ")
    if not validate_weight(weight):
        return "Please enter a valid weight."

    height = input("Enter your height in inches: ")
    try:
        height = int(height)
        if height <= 0:
            raise ValueError
    except ValueError:
        return "Please enter a valid height."

    age = input("Enter your age in years: ")
    try:
        age = int(age)
        if age <= 0:
            raise ValueError
    except ValueError:
        return "Please enter a valid age."

    gender_choice = get_gender_choice()
    gender = input(f"Enter your gender ({'/'.join([f'{k} for {v}' for k, v in gender_choice.items()])}): ")
    if int(gender) not in gender_choice:
        return "Invalid gender selection."

    activity_levels = {
        1: 'Sedentary (little or no exercise)',
        2: 'Lightly active (light exercise/sports 1-3 days/week)',
        3: 'Moderately active (moderate exercise/sports 3-5 days/week)',
        4: 'Very active (hard exercise/sports 6-7 days a week)',
        5: 'Extra active (very hard exercise/sports & physical job or 2x training)'
    }
    activity_level = input(f"Select your activity level ({'/'.join([f'{k} for {v}' for k, v in activity_levels.items()])}): ")
    if int(activity_level) not in activity_levels:
        return "Invalid activity level selection."

    dietary_restrictions = get_dietary_restrictions()
    diet = input(f"Select your dietary restriction ({'/'.join([f'{k} for {v}' for k, v in dietary_restrictions.items()])}): ")
    if int(diet) not in dietary_restrictions:
        return "Invalid dietary restriction selection."
    if int(diet) == 8:
        other_diet = input("Please describe your dietary restriction: ")

    goal_choice = get_goal()
    goal = input(f"What is your goal? ({'/'.join([f'{k} for {v}' for k, v in goal_choice.items()])}): ")
    weight_goal = None
    if int(goal) in [1, 2]:
        weight_goal = input("Enter your weight goal in lbs (e.g., 5 for 5 lbs): ")
        try:
            weight_goal = float(weight_goal)
            if weight_goal <= 0:
                raise ValueError("Please enter a positive number.")
        except ValueError as e:
            return str(e)

    return {
        "weight": float(weight),
        "height": height,
        "age": age,
        "gender": gender_choice[int(gender)],
        "activity_level": int(activity_level),
        "dietary_restriction": dietary_restrictions[int(diet)] if int(diet) != 7 else other_diet,
        "goal": goal_choice[int(goal)],
        "weight_goal": weight_goal
    }

def calculate_daily_calories(user_data):
    if user_data['gender'] == 'Male':
        bmr = 10 * (user_data['weight'] / 2.2) + 6.25 * (user_data['height'] * 2.54) - 5 * user_data['age'] + 5
    else:
        bmr = 10 * (user_data['weight'] / 2.2) + 6.25 * (user_data['height'] * 2.54) - 5 * user_data['age'] - 161

    activity_multiplier = [1.2, 1.375, 1.55, 1.725, 1.9][user_data['activity_level'] - 1]
    daily_calories = bmr * activity_multiplier
    print
    return daily_calories

In [25]:
def generate_recipe_descriptions(user_data):
    prompt = (
        f"Based on the following user profile, generate five healthy and accessible meal "
        f"recipes that align with their dietary restriction ({user_data['dietary_restriction']}), "
        f"goal ({user_data['goal']}), and daily calorie intake ({calculate_daily_calories(user_data)} calories): "
        f"\n\nUser Profile:\n"
        f"- Weight: {user_data['weight']} lbs\n"
        f"- Height: {user_data['height']} inches\n"
        f"- Age: {user_data['age']}\n"
        f"- Gender: {user_data['gender']}\n"
        f"- Activity Level: {user_data['activity_level']}\n"
        f"- Dietary Restriction: {user_data['dietary_restriction']}\n"
        f"- Goal: {user_data['goal']}\n"
        f"- Daily Calorie Intake: {calculate_daily_calories(user_data)} calories\n\n"
          f"Each recipe description should just be a brief description."
    )
    return generate(prompt)


In [26]:
def generate_detailed_recipe(user_data, selected_recipe):
    prompt = (
        f"Write a detailed recipe for '{selected_recipe}' suitable for someone with the following profile "
        f"and goals. The recipe should align with their dietary restriction ({user_data['dietary_restriction']}), "
        f"goal ({user_data['goal']}), and should fit into their daily calorie intake ({calculate_daily_calories(user_data)} calories). "
        f"Include step-by-step instructions, serving size, and nutritional information per serving.\n\n"
        f"User Profile:\n"
        f"- Dietary Restriction: {user_data['dietary_restriction']}\n"
        f"- Goal: {user_data['goal']}\n"
        f"- Daily Calorie Intake: {calculate_daily_calories(user_data)} calories"
    )
    return generate(prompt)


In [ ]:
# cook time
# input ingredients you have
# shopping list

In [27]:
def generate_grocery_list(ingredients_on_hand, detailed_recipe):
    prompt = (
        f"Given that the user already has the following ingredients at home: {', '.join(ingredients_on_hand)}, "
        f"and based on the detailed recipe below, generate a grocery list for the remaining ingredients needed.\n\n"
        f"Detailed Recipe:\n{detailed_recipe}\n\n"
        f"List only the ingredients that are not already present in the user's kitchen."
    )
    return generate(prompt)


In [36]:
def main():
    user_data = input_user_data()
    if isinstance(user_data, str):
        print(user_data)
        return

    print("User data collected successfully:", user_data)

    recipe_descriptions_prompt = generate_recipe_descriptions(user_data)
    print("\nFetching recipe suggestions...\n")
    recipe_descriptions_response = generate(recipe_descriptions_prompt)
    recipe_descriptions = recipe_descriptions_response.split('\n')
    print("\n".join(recipe_descriptions))

    selected_recipe_index = input("\nPlease select one of the above recipes by number (1-5): ")
    try:
        selected_recipe_index = int(selected_recipe_index) - 1
        if selected_recipe_index not in range(len(recipe_descriptions)):
            raise ValueError
    except ValueError:
        print("Invalid selection. Please enter a number between 1 and 5.")
        return

    selected_recipe = recipe_descriptions[selected_recipe_index]
    print(f"\nYou selected: {selected_recipe}\n")

    detailed_recipe_prompt = generate_detailed_recipe(user_data, selected_recipe)
    print("\nGenerating detailed recipe...\n")
    detailed_recipe_response = generate(detailed_recipe_prompt)
    detailed_recipe = detailed_recipe_response.strip()
    print(detailed_recipe)

    ingredients_on_hand = input("\nEnter the ingredients you already have, separated by commas: ").split(',')
    ingredients_on_hand = [ingredient.strip() for ingredient in ingredients_on_hand]

    grocery_list_prompt = generate_grocery_list(ingredients_on_hand, detailed_recipe)
    print("\nGenerating grocery list...\n")
    grocery_list_response = generate(grocery_list_prompt)
    grocery_list = grocery_list_response.strip()
    print(grocery_list)


In [37]:
main()

Enter your weight in lbs: 120
Enter your height in inches: 66
Enter your age in years: 20
Enter your gender (1 for Male/2 for Female): 2
Select your activity level (1 for Sedentary (little or no exercise)/2 for Lightly active (light exercise/sports 1-3 days/week)/3 for Moderately active (moderate exercise/sports 3-5 days/week)/4 for Very active (hard exercise/sports 6-7 days a week)/5 for Extra active (very hard exercise/sports & physical job or 2x training)): 2
Select your dietary restriction (1 for Keto/2 for Halal/3 for Vegan/4 for Vegetarian/5 for Gluten-Free/6 for Paleo/7 for Other): 1
What is your goal? (1 for Gain Weight/2 for Lose Weight/3 for Maintain Weight): 2
Enter your weight goal in lbs (e.g., 5 for 5 lbs): 5
User data collected successfully: {'weight': 120.0, 'height': 66, 'age': 20, 'gender': 'Female', 'activity_level': 2, 'dietary_restriction': 'Keto', 'goal': 'Lose Weight', 'weight_goal': 5.0}

Fetching recipe suggestions...

6. Recipe: Keto Broccoli Cheese Soup
   In